<a href="https://colab.research.google.com/github/YoshiyukiKono/langchain_for_beginners/blob/main/01_RAG_pdf.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# PDFファイルを使ったRAG

https://python.langchain.com/docs/modules/data_connection/document_loaders/pdf

## 準備

### Astra DBの準備
 1. Astraで、新しいベクトル検索対応データベースを作成します。
 2. トークンとAPIエンドポイントを取得します。


### パッケージインストール

In [2]:
!pip install ragstack-ai==0.7.0

In [7]:
!pip show langchain
!pip show openai
!pip show tiktoken
!pip show astrapy

Name: langchain
Version: 0.1.4
Summary: Building applications with LLMs through composability
Home-page: https://github.com/langchain-ai/langchain
Author: 
Author-email: 
License: MIT
Location: /usr/local/lib/python3.10/dist-packages
Requires: aiohttp, async-timeout, dataclasses-json, jsonpatch, langchain-community, langchain-core, langsmith, numpy, pydantic, PyYAML, requests, SQLAlchemy, tenacity
Required-by: ragstack-ai
Name: openai
Version: 1.12.0
Summary: The official Python library for the openai API
Home-page: 
Author: 
Author-email: OpenAI <support@openai.com>
License: 
Location: /usr/local/lib/python3.10/dist-packages
Requires: anyio, distro, httpx, pydantic, sniffio, tqdm, typing-extensions
Required-by: langchain-openai, llama-index
Name: tiktoken
Version: 0.5.2
Summary: tiktoken is a fast BPE tokeniser for use with OpenAI's models
Home-page: 
Author: Shantanu Jain
Author-email: shantanu@openai.com
License: MIT License
        
        Copyright (c) 2022 OpenAI, Shantanu Jain


In [8]:
!pip install pypdf==3.17.0

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 277.4/277.4 kB 4.8 MB/s eta 0:00:00


## OpenAI 接続

In [9]:
import os
import getpass

os.environ["OPENAI_API_KEY"] = getpass.getpass()

··········


入力されたキーによる接続確認

*【下記セルを実行すると入力欄が表示されます】*

In [10]:
import openai
from openai import OpenAI

client = OpenAI()
client.models.list()

SyncPage[Model](data=[Model(id='gpt-4-vision-preview', created=1698894917, object='model', owned_by='system'), Model(id='dall-e-3', created=1698785189, object='model', owned_by='system'), Model(id='gpt-4-turbo-preview', created=1706037777, object='model', owned_by='system'), Model(id='gpt-3.5-turbo-0613', created=1686587434, object='model', owned_by='openai'), Model(id='text-embedding-3-large', created=1705953180, object='model', owned_by='system'), Model(id='dall-e-2', created=1698798177, object='model', owned_by='system'), Model(id='gpt-3.5-turbo-instruct-0914', created=1694122472, object='model', owned_by='system'), Model(id='whisper-1', created=1677532384, object='model', owned_by='openai-internal'), Model(id='tts-1-hd-1106', created=1699053533, object='model', owned_by='system'), Model(id='tts-1-hd', created=1699046015, object='model', owned_by='system'), Model(id='babbage-002', created=1692634615, object='model', owned_by='system'), Model(id='gpt-3.5-turbo-instruct', created=1692

## Astra DB接続
*【下記セルを実行すると入力欄が表示されます】*

AstraCS:から始まるトークンを設定します。

In [11]:
import getpass

YOUR_TOKEN = getpass.getpass()

··········


*【下記セルを実行すると入力欄が表示されます】*

あなたのAstra環境のAPIエンドポイントの値をコピーして、設定します。

In [12]:
API_ENDPOINT = input()

https://1014346a-a40c-4d1a-b1a3-78769cc72312-us-east1.apps.astra.datastax.com


接続確認

In [27]:
from astrapy.db import AstraDB

db = AstraDB(
  token=YOUR_TOKEN,
  api_endpoint=API_ENDPOINT)

print(f"Connected to Astra DB: {db.get_collections()}")

Connected to Astra DB: {'status': {'collections': ['langchain_message_store', 'semantics', 'semantics_ragstack', 'vector_context_datastax']}}


## LangChainの利用



In [ ]:
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores import Cassandra
from langchain.document_loaders import TextLoader

### PDFファイルの準備

このリポジトリの`data`フォルダにあるファイル(`DataStax_Vector-Search.pdf`)を、Colab環境にアップロードしてください。

このアップロードしたファイルを下記で読み込みます。

In [14]:
from langchain.document_loaders import PyPDFLoader

loader = PyPDFLoader("./DataStax_Vector-Search.pdf")
pages = loader.load_and_split()

In [15]:
print(len(pages))
print(pages[0])
print(pages[len(pages)-1])

24
page_content='ホワイトペーパー\n生成 AIアプリ\nのための\nベクトル検索\nAIアプリケーション開発にベクトル検索を活用す\nるための開発者/アーキテクト向けガイド\nこのドキュメントは、生成 AIアプリケーションを企業独\n自のデータと組み合わせて、設計・構築しようとしてい\nる全ての方のためのガイドです。組織が理解すべき重\n要な概念と考慮事項を取り扱うだけでなく、ベクトル検\n索を用いて、 LLMの持つ機能を大幅に拡張するため\nのシンプルで強力なアプローチについても解説しま\nす。' metadata={'source': './DataStax_Vector-Search.pdf', 'page': 0}
page_content='20. Karpathy, Andrej. “Andrej Karpathy on Twitter: "The hottest new programming language is\nEnglish."” Twitter , 2023年 1月24日 ,https://twitter .com/karpathy/status/1617979122625712128\n21. “MTEB Leaderboard - a Hugging Face Space by mteb.” Hugging Face,\nhttps://huggingface.co/spaces/mteb/leaderboard\n22.https://platform.openai.com/docs/guides/embeddings/types-of-embedding-models\n23. Instructor Text Embedding, https://instructor-embedding.github.io/\n24. “Libraries - OpenAI API.” Platform OpenAI, https://platform.openai.com/docs/libraries\n25. “Vertex AI.” Google Cloud, https://cloud.google.com/vertex-ai\n©2023 DataStax Inc.、全著作権所有。 DataStax は、米国およびその他

`RecursiveCharacterTextSplitter`を使って、より細かいチャンクに分割する。

In [16]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(chunk_size=400, chunk_overlap=0)

texts = text_splitter.split_documents(pages)

In [18]:
print(len(texts))
print(texts[0])
print(texts[len(texts)-1])

82
page_content='ホワイトペーパー\n生成 AIアプリ\nのための\nベクトル検索\nAIアプリケーション開発にベクトル検索を活用す\nるための開発者/アーキテクト向けガイド\nこのドキュメントは、生成 AIアプリケーションを企業独\n自のデータと組み合わせて、設計・構築しようとしてい\nる全ての方のためのガイドです。組織が理解すべき重\n要な概念と考慮事項を取り扱うだけでなく、ベクトル検\n索を用いて、 LLMの持つ機能を大幅に拡張するため\nのシンプルで強力なアプローチについても解説しま\nす。' metadata={'source': './DataStax_Vector-Search.pdf', 'page': 0}
page_content='23. Instructor Text Embedding, https://instructor-embedding.github.io/\n24. “Libraries - OpenAI API.” Platform OpenAI, https://platform.openai.com/docs/libraries\n25. “Vertex AI.” Google Cloud, https://cloud.google.com/vertex-ai\n©2023 DataStax Inc.、全著作権所有。 DataStax は、米国およびその他の国における DataStax, Inc. およびそ\nの子会社の登録商標です。\n24' metadata={'source': './DataStax_Vector-Search.pdf', 'page': 23}


In [29]:
docs=texts

## ベクトルストアの構築

OpenAIのエンベディングモデルではなく、（連続）利用制限のない、オープンなエンベディングモデルを利用します。

In [21]:
!pip install sentence_transformers==2.2.2

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 1.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for sentence_transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125923 sha256=e356960757d668336a516051326fd63fd5036c7808e8b81ed4f785f47ba9da86
  Stored in directory: /root/.cache/pip/wheels/62/f2/10/1e606fd5f02395388f74e7462910fe851042f97238cbbd902f
Successfully built sentence_transformers


In [22]:
from langchain.embeddings import HuggingFaceEmbeddings

embeddings = HuggingFaceEmbeddings()

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


.gitattributes:   0%|          | 0.00/1.18k [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.6k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/571 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

train_script.py:   0%|          | 0.00/13.1k [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/torch/_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()


`embed_documents`メソッドの利用。複数の文字列を配列で受け取る。ベクトルDBに登録する複数のデータをまとめてエンベッドする。

In [23]:
text = "テスト文字列"

doc_result = embeddings.embed_documents([text])

print(len(doc_result[0]))
print(doc_result)

768
[[0.02339586801826954, -0.051528092473745346, -0.01799503155052662, 0.010700330138206482, 0.039436064660549164, 0.027511082589626312, -0.03647975996136665, 0.06465698033571243, 0.013887857086956501, 0.018204664811491966, 0.03117774985730648, 0.015076836571097374, 0.07378007471561432, 0.029201000928878784, -0.043200548738241196, -0.07707522064447403, -0.0067560975439846516, 0.0345374271273613, 0.045760996639728546, -0.02631659060716629, 0.015634726732969284, 0.012039588764309883, 0.015451109036803246, -0.014117742888629436, 0.015340771526098251, -0.044638700783252716, 0.012640316039323807, -0.012221626937389374, 0.029994942247867584, 0.0002732427674345672, 0.047916773706674576, -0.022249307483434677, -0.0039941188879311085, 0.008768660947680473, 1.510015181338531e-06, -0.017982913181185722, -0.021689971908926964, -0.026938535273075104, -0.018404435366392136, -0.04279183968901634, 0.041685618460178375, -0.0442582331597805, -0.006897759158164263, 0.016141779720783234, 0.02712154760956

`embed_query`メソッドの利用。一つの文字列を配列で受け取る。ベクトルDBへのクエリに使う文字列をエンベッドする。

In [24]:
query_result = embeddings.embed_query(text)

print(len(query_result))
print(query_result)

768
[0.02339586801826954, -0.051528092473745346, -0.01799503155052662, 0.010700330138206482, 0.039436064660549164, 0.027511082589626312, -0.03647975996136665, 0.06465698033571243, 0.013887857086956501, 0.018204664811491966, 0.03117774985730648, 0.015076836571097374, 0.07378007471561432, 0.029201000928878784, -0.043200548738241196, -0.07707522064447403, -0.0067560975439846516, 0.0345374271273613, 0.045760996639728546, -0.02631659060716629, 0.015634726732969284, 0.012039588764309883, 0.015451109036803246, -0.014117742888629436, 0.015340771526098251, -0.044638700783252716, 0.012640316039323807, -0.012221626937389374, 0.029994942247867584, 0.0002732427674345672, 0.047916773706674576, -0.022249307483434677, -0.0039941188879311085, 0.008768660947680473, 1.510015181338531e-06, -0.017982913181185722, -0.021689971908926964, -0.026938535273075104, -0.018404435366392136, -0.04279183968901634, 0.041685618460178375, -0.0442582331597805, -0.006897759158164263, 0.016141779720783234, 0.027121547609567

In [28]:
from langchain_community.vectorstores import AstraDB
vstore = AstraDB(
    embedding=embeddings,
    collection_name="pdf_collection",
    token=YOUR_TOKEN,
    api_endpoint=API_ENDPOINT,
)


上記の操作により、下記のテーブルとインデックスが作成されます。
```
CREATE TABLE default_keyspace.pdf_collection (
    key frozen<tuple<tinyint, text>> PRIMARY KEY,
    array_contains set<text>,
    array_size map<text, int>,
    doc_json text,
    exist_keys set<text>,
    query_bool_values map<text, tinyint>,
    query_dbl_values map<text, decimal>,
    query_null_values set<text>,
    query_text_values map<text, text>,
    query_timestamp_values map<text, timestamp>,
    query_vector_value vector<float, 768>,
    tx_id timeuuid
) WITH additional_write_policy = '99p'
    AND bloom_filter_fp_chance = 0.01
    AND caching = {'keys': 'ALL', 'rows_per_partition': 'NONE'}
    AND comment = ''
    AND compaction = {'class': 'org.apache.cassandra.db.compaction.UnifiedCompactionStrategy'}
    AND compression = {'chunk_length_in_kb': '16', 'class': 'org.apache.cassandra.io.compress.LZ4Compressor'}
    AND crc_check_chance = 1.0
    AND default_time_to_live = 0
    AND gc_grace_seconds = 864000
    AND max_index_interval = 2048
    AND memtable_flush_period_in_ms = 0
    AND min_index_interval = 128
    AND read_repair = 'BLOCKING'
    AND speculative_retry = '99p';
CREATE CUSTOM INDEX pdf_collection_array_contains ON default_keyspace.pdf_collection (values(array_contains)) USING 'StorageAttachedIndex';
CREATE CUSTOM INDEX pdf_collection_array_size ON default_keyspace.pdf_collection (entries(array_size)) USING 'StorageAttachedIndex';
CREATE CUSTOM INDEX pdf_collection_exists_keys ON default_keyspace.pdf_collection (values(exist_keys)) USING 'StorageAttachedIndex';
CREATE CUSTOM INDEX pdf_collection_query_bool_values ON default_keyspace.pdf_collection (entries(query_bool_values)) USING 'StorageAttachedIndex';
CREATE CUSTOM INDEX pdf_collection_query_dbl_values ON default_keyspace.pdf_collection (entries(query_dbl_values)) USING 'StorageAttachedIndex';
CREATE CUSTOM INDEX pdf_collection_query_null_values ON default_keyspace.pdf_collection (values(query_null_values)) USING 'StorageAttachedIndex';
CREATE CUSTOM INDEX pdf_collection_query_text_values ON default_keyspace.pdf_collection (entries(query_text_values)) USING 'StorageAttachedIndex';
CREATE CUSTOM INDEX pdf_collection_query_timestamp_values ON default_keyspace.pdf_collection (entries(query_timestamp_values)) USING 'StorageAttachedIndex';
CREATE CUSTOM INDEX pdf_collection_query_vector_value ON default_keyspace.pdf_collection (query_vector_value) USING 'StorageAttachedIndex' WITH OPTIONS = {'similarity_function': 'cosine'};
```

Astra DBの「CQL Console」タブで、次のコマンドを実行して上記のDDLを確認することができます。

```
desc default_keyspace.pdf_collection;
```


In [30]:
print(len(docs))

82


In [35]:
inserted_ids = vstore.add_documents(docs)
print(f"\nInserted {len(inserted_ids)} documents.")


Inserted 82 documents.



CQLコンソール上で、下記のように、登録されたデータの件数を確認することができます。

```
select count(key) from default_keyspace.pdf_collection;
```

```
token@cqlsh:default_keyspace> select count(key) from default_keyspace.pdf_collection;

 system.count(key)
-------------------
                82

(1 rows)
```

まず、以下では、Astra DBベクトルデータベースに対する、直接のベクトル検索実行を試してみます。

In [41]:
query = "Cassandra"
docs = vstore.similarity_search(query)
print(len(docs))
for doc in docs:
  print(doc)

4
page_content='分散データベース Apache Cassandraの拡張機能であるため、ベクトルとともに、そのベクトルに関連する\nメタデータを保存しておくことも容易であり、ベクトル検索のみではなく、メタデータを使った操作との組み合\nわせについても、利点を持っています。このような大規模データ利用のための信頼性・スケールと、データ\nベースとしての汎用性は、 AstraDBの持つ、単なるベクトルデータベースとは異なる利点です。\n12' metadata={'source': './DataStax_Vector-Search.pdf', 'page': 11}
page_content='より高度な、広く用いられている戦略として、ドキュメントの、または言語の構造に基づいたチャンク化があ\nります。例としては、ドキュメントのセクション、段落ごと、またはセンテンスの境界を基準にしたチャンク化が\n挙げられます。\nドキュメントのチャンクのセットを作成したら、各チャンクのベクトルを、先の手順で定義した関連メタデータと\n共に、データストアに保存します。これにより、近似最近傍探索（ ANN: Approximate Nearest Neighbor\n）を利用した高速ベクトル検索が可能になります。 DataStaxの Apache Cassandraマネージドサービスで\nあるAstraDB は、ベクトル検索の機能を有しており、大規模なベクトルデータセットを操作する場合の優れ\nた選択肢となります。そしてまた、 AstraDBのベクトル検索機能は、高い可用性と柔軟なスケールを有した' metadata={'source': './DataStax_Vector-Search.pdf', 'page': 11}
page_content='https://www.anthropic.com/index/anthropic-partners-with-google-cloud\n5. “Google AI PaLM 2 – Google AI.” Google AI, https://ai.google/discover/palm2/\n6. Try Bard, an AI experiment by Google, https://bard.google.c

## Agent

LangChainのエージェントを試してみます。

In [42]:
retriever = vstore.as_retriever(search_kwargs={"k":1})

In [43]:

from langchain.agents.agent_toolkits import create_retriever_tool, create_conversational_retrieval_agent
from langchain.chat_models import ChatOpenAI
from langchain.embeddings import OpenAIEmbeddings
from langchain.callbacks import StreamlitCallbackHandler
from langchain.schema import BaseRetriever, Document, SystemMessage

In [44]:
from langchain.globals import set_debug, set_verbose

set_debug(True)
set_verbose(False)

In [45]:
from langchain.llms import OpenAI
from langchain.chains import LLMMathChain
from langchain.agents import Tool
from langchain.agents import load_tools
from langchain.agents import initialize_agent
from langchain.agents import AgentType

llm = OpenAI(model_name="gpt-3.5-turbo-instruct")

retriever_tool = create_retriever_tool(
        retriever, "my_retrevier", "Useful when searching for the information about technologies")
tools=[retriever_tool]

system_message_content = """
    Please answer using the given my_retreiver.
    """
system_message_content = f"{system_message_content} All the responses should be in Japanese language."
system_message = SystemMessage(content=system_message_content)

agent = initialize_agent(tools, llm, agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION, system_message=system_message, verbose=True, max_tokens_limit=500)

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The class `langchain_community.llms.openai.OpenAI` was deprecated in langchain-community 0.0.10 and will be removed in 0.2.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import OpenAI`.
  warn_deprecated(
/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The function `initialize_agent` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use Use new agent constructor methods like create_react_agent, create_json_agent, create_structured_chat_agent, etc. instead.
  warn_deprecated(


In [52]:
agent.run("AstraDBについて説明して")

[chain/start] [1:chain:AgentExecutor] Entering Chain run with input:
{
  "input": "AstraDBについて説明して"
}
[chain/start] [1:chain:AgentExecutor > 2:chain:LLMChain] Entering Chain run with input:
{
  "input": "AstraDBについて説明して",
  "agent_scratchpad": "",
  "stop": [
    "\nObservation:",
    "\n\tObservation:"
  ]
}
[llm/start] [1:chain:AgentExecutor > 2:chain:LLMChain > 3:llm:OpenAI] Entering LLM run with input:
{
  "prompts": [
    "Answer the following questions as best you can. You have access to the following tools:\n\nmy_retrevier: Useful when searching for the information about technologies\n\nUse the following format:\n\nQuestion: the input question you must answer\nThought: you should always think about what to do\nAction: the action to take, should be one of [my_retrevier]\nAction Input: the input to the action\nObservation: the result of the action\n... (this Thought/Action/Action Input/Observation can repeat N times)\nThought: I now know the final answer\nFinal Answer: the final a

'分散データベース Apache Cassandraの拡張機能として、ベクトルデータベース AstraDBは、ベクトル検索だけでなく、メタデータを使った操作との組み合わせにも利点を持ち、大規模データ利用の信頼性・スケールと、データベースとしての汎用性を兼ね備えている。'